# PART3: Multilabel-Classification using scikit-learn Linear models

## Load Data

In [3]:
import numpy as np
import pandas as pd 

data_root = '/media/sf_Yelp/input/'

train_photos = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_photo_to_biz = pd.read_csv(data_root+'train_photo_to_biz_ids.csv', index_col='photo_id')

train_df = pd.read_csv(data_root+"train_biz_fc7features.csv")
test_df  = pd.read_csv(data_root+"test_biz_fc7features.csv")

y_train = train_df['label'].values
X_train = train_df['feature vector'].values
X_test = test_df['feature vector'].values

def convert_label_to_array(str_label):
    str_label = str_label[1:-1]
    str_label = str_label.split(',')
    return [int(x) for x in str_label if len(x)>0]

def convert_feature_to_vector(str_feature):
    str_feature = str_feature[1:-1]
    str_feature = str_feature.split(',')
    return [float(x) for x in str_feature]

y_train = np.array([convert_label_to_array(y) for y in train_df['label']])
X_train = np.array([convert_feature_to_vector(x) for x in train_df['feature vector']])
X_test = np.array([convert_feature_to_vector(x) for x in test_df['feature vector']])


In [9]:
print "X_train: ", X_train.shape
print "y_train: ", y_train.shape
print "X_test: ", X_test.shape
print "train_df:"
train_df[0:5]

X_train:  (1996, 4096)
y_train:  (1996,)
X_test:  (10000, 4096)
train_df:


,business,label,feature vector
0,1000%,"(1, 2, 3, 4, 5, 6, 7)","[0.20032024, 0.44084537, 0.23249489, 0.3600976..."
1,1001%,"(0, 1, 6, 8)","[0.0013769998, 0.59398097, 0.55060995, 0.18394..."
2,100%,"(1, 2, 4, 5, 6, 7)","[0.11435749, 0.033177156, 0.12572332, 0.539482..."
3,1006%,"(1, 2, 4, 5, 6)","[0.075851507, 0.052600037, 0.059594199, 0.7067..."
4,1010%,"(0, 6, 8)","[0.39024171, 0.28424361, 0.0, 0.1655795, 0.460..."


## Train a SVM using cross-validation and assess performance(F1-score)

In [56]:
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import xgboost as xgb
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.metrics import f1_score

import time
t=time.time()

mlb = MultiLabelBinarizer()
y_ptrain= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix

SEED = 0
random_state = np.random.RandomState(SEED)
#X_ptrain, X_ptest, y_ptrain, y_ptest = train_test_split(X_train, y_ptrain, test_size=.2,random_state=random_state)
# rfc = RandomForestClassifier(n_estimators=1000, criterion='entropy', max_depth=4,
#                                 min_samples_split=4, min_samples_leaf=3,
#                                 max_features='auto', bootstrap=True,
#                                 oob_score=True, n_jobs=-1,
#                                 random_state=SEED, verbose=0)
classifier = OneVsRestClassifier(linear_model.LinearRegression())
#classifier.fit(X_ptrain, y_ptrain)
#y_ppredict = classifier.predict(X_ptest)
y_ppredict = cross_validation.cross_val_predict(classifier, X_train, y_ptrain, cv=2, verbose =1)

print "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

Time passed:  42.2 sec


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   42.1s finished


In [57]:
print "Samples of predicted labels (in binary matrix):\n", y_ppredict[0:3]
print "\nSamples of predicted labels:\n", mlb.inverse_transform(y_ppredict[0:3])

Samples of predicted labels (in binary matrix):
[[0 1 1 1 1 1 1 1 0]
 [0 1 1 0 0 1 1 1 0]
 [0 1 1 0 1 1 1 1 0]]

Samples of predicted labels:
[(1, 2, 3, 4, 5, 6, 7), (1, 2, 5, 6, 7), (1, 2, 4, 5, 6, 7)]


In [58]:
statistics = pd.DataFrame(columns=[ "attribuite "+str(i) for i in range(9)]+['num_biz'], index = ["biz count", "biz ratio"])
statistics.loc["biz count"] = np.append(np.sum(y_ppredict, axis=0), len(y_ppredict))
pd.options.display.float_format = '{:.0f}%'.format
statistics.loc["biz ratio"] = statistics.loc["biz count"]*100/len(y_ppredict) 
statistics

,attribuite 0,attribuite 1,attribuite 2,attribuite 3,attribuite 4,attribuite 5,attribuite 6,attribuite 7,attribuite 8,num_biz
biz count,622,1000,1059,1112,624,1218,1393,642,1153,1996
biz ratio,31%,50%,53%,56%,31%,61%,70%,32%,58%,100%


In [59]:
from sklearn.metrics import f1_score

print "F1 score: ", f1_score(y_ptrain, y_ppredict, average='micro') 
print "Individual Class F1 score: ", f1_score(y_ptrain, y_ppredict, average=None)

F1 score:  0.762498569958
Individual Class F1 score:  [ 0.58932715  0.7676869   0.80767386  0.60330969  0.68488471  0.83096879
  0.89865601  0.66721582  0.81221246]


## Re-Train a SVM using all training data, and make predictions on test set

In [10]:
## Uncomment if skip previous train
from sklearn import svm
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import time
import xgboost as xgb
from sklearn.svm import LinearSVC
from sklearn import linear_model
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import AdaBoostClassifier

t = time.time()

mlb = MultiLabelBinarizer()
y_train= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix

SEED = 0
random_state = np.random.RandomState(SEED)
classifier = OneVsRestClassifier(linear_model.PassiveAggressiveClassifier(n_iter=50))
classifier.fit(X_train, y_train)

y_predict = classifier.predict(X_test)
#y_predict = classifier.predict_proba(X_test)
np.savetxt(data_root+"../submissions/ensemble/y_predict_0.812068670507_PAC_submission_fc7_4096.csv", y_predict, delimiter=",")
#y_predict = y_predict > 0.5

#print list(mlb.classes_)
y_predict_label = mlb.inverse_transform(y_predict) #Convert binary matrix back to labels

print "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

Time passed:  9.8 sec


In [11]:
X_test.shape

(10000, 4096)

In [12]:
test_data_frame  = pd.read_csv(data_root+"test_biz_fc7features.csv")
df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(test_data_frame)):
    biz = test_data_frame.loc[i]['business']
    label = y_predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    df.loc[i] = [str(biz), label]
    
    
with open(data_root+"../submissions/0.812068670507_PAC_submission_fc7_4096.csv",'w') as f:
    df.to_csv(f, index=False)    
    

In [13]:
statistics = pd.DataFrame(columns=[ "attribuite "+str(i) for i in range(9)]+['num_biz'], index = ["biz count", "biz ratio"])
statistics.loc["biz count"] = np.append(np.sum(y_predict, axis=0), len(y_predict))
pd.options.display.float_format = '{:.0f}%'.format
statistics.loc["biz ratio"] = statistics.loc["biz count"]*100/len(y_predict) 
statistics

,attribuite 0,attribuite 1,attribuite 2,attribuite 3,attribuite 4,attribuite 5,attribuite 6,attribuite 7,attribuite 8,num_biz
biz count,2889,4452,7257,3035,1121,9274,9136,2265,8110,10000
biz ratio,29%,45%,73%,30%,11%,93%,91%,23%,81%,100%


## Ensemble of previous models

In [4]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import MultiLabelBinarizer

data_root = '/media/sf_Yelp/input/'
mlb = MultiLabelBinarizer()
y_train= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix


y_predict_BR  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.828991940194_BayesianRidge__submission_fc7_4096.csv",header=None)
y_predict_LoR  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.824536875218_LogisticRegressionCV_submission_fc7_4096.csv",header=None)
y_predict_XGB  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.823321762664_XGB_d4_n500_submission_fc7_4096.csv",header=None)
y_predict_RF  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.807488855869_RF_d4_n1000_submission_fc7_4096.csv",header=None)
y_predict_RCcv  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.825110002933_RCcv_norm_submission_fc7_4096.csv",header=None)
y_predict_RC  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.820068600663_RC_norm_submission_fc7_4096.csv",header=None)
y_predict_PAC  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.812068670507_PAC_submission_fc7_4096.csv",header=None)
#y_predict_LR  = pd.read_csv(data_root+"../submissions/ensemble/y_predict_0.762498569958_LR_submission_fc7_4096.csv",header=None)


ensemble_predict = (y_predict_BR + y_predict_LoR + y_predict_XGB + y_predict_RF+ y_predict_RCcv+ y_predict_RC+ y_predict_PAC)/7
ensemble_predict = ensemble_predict >0.5
ensemble_predict = pd.DataFrame.as_matrix(ensemble_predict)
print "Samples of ensemble predict labels (in binary matrix):\n", ensemble_predict[0:3]
#print list(mlb.classes_)
y_predict_label = mlb.inverse_transform(ensemble_predict) #Convert binary matrix back to labels
print "Samples of y_predict_label (in binary matrix):\n", y_predict_label[0:3]


test_data_frame  = pd.read_csv(data_root+"test_biz_fc7features.csv")
df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(test_data_frame)):
    biz = test_data_frame.loc[i]['business']
    label = y_predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    df.loc[i] = [str(biz), label]

with open(data_root+"../submissions/ensemble_7models_submission_fc7_4096.csv",'w') as f:
    df.to_csv(f, index=False)  
    

Samples of ensemble predict labels (in binary matrix):
[[False  True  True  True False  True  True False False]
 [False  True  True  True False  True  True False  True]
 [False  True False  True False  True  True False  True]]
Samples of y_predict_label (in binary matrix):
[(1, 2, 3, 5, 6), (1, 2, 3, 5, 6, 8), (1, 3, 5, 6, 8)]
